In [1]:
#Importing dependencies.
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import time
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, PrimaryKeyConstraint, func,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
import sqlite3
from sqlalchemy.ext.automap import automap_base

#Use Splinter to establish a browser that connects to the pickleheads website.
browser = Browser('chrome')
url = 'https://www.pickleheads.com/courts/us/florida'

#Visited the website and clicked on the button 'Show More'.
browser.visit(url)
buttons = browser.find_by_css(".css-2ttlle")
button_index_to_click = 1
if button_index_to_click < len(buttons):
    button_to_click = buttons[button_index_to_click]
    button_to_click.click()
    time.sleep(2)
html = browser.html

#Create a Beautiful Soup object from the html returned from the browser and parse the information.
soup = BeautifulSoup(html, 'html.parser')

In [2]:
#Find the section of interest that has the class css-12eoos1 and save it as a variable.
whole_list=soup.find('div',class_='css-12eoos1')

In [3]:
#Set up three empty lists to append to.
location = []
city = []
courts=[]

#Using a for loop, find the location name, city name and number of courts and append them to their respective lists.
for entry in whole_list:
    location_data = entry.find('span', class_='css-uyrttv')
    if location_data:
        location.append(location_data.text)
    else:
        continue

    city_data = entry.find(class_='css-1vl3ez2')
    
    if city_data:
        city.append(city_data.text)

    court_data=entry.select('.css-1vl3ez2')
    if court_data: 
        last_span=court_data[-1] 
        courts.append(last_span.text)

In [13]:
#Create a dataframe with the locations and cities using the three lists from above.
pickleball_dict={"Location Name":location,
                 "City":city,
                 "Number of Courts":courts}
fl_pickleball_df=pd.DataFrame(pickleball_dict)

#Sorted the pickleball dataframe by city.
fl_pickleball_df_sorted=fl_pickleball_df.sort_values(by='City',ascending=True)

#Read in a csv from https://github.com/kelvins/US-Cities-Database/blob/main/csv/us_cities.csv after adding missing cities.
#Used https://www.statsamerica.org/CityCountyFinder/ to fill in missing cities and clear up cities with two counties listed.
#Also used google when a city was not in the CityCountyFinder and used google to look up missing latitudes and longitudes.
city_county_df=pd.read_csv('Florida City County and Coordinates.csv')

#Formatted the names in the dataframe so it would match the dataframe it will be merged with.
city_county_df=city_county_df.rename(columns={"CITY":"City","COUNTY":"County","LATITUDE":"Latitude","LONGITUDE":"Longitude"})

In [14]:
# Merged the pickleball dataframe with the city county dataframe to enhance the pickleball information dataframe with coordinates and counties.
fl_full_info=pd.merge(fl_pickleball_df_sorted,city_county_df,on='City',how='left')
fl_full_info.to_csv('pickleball_courts_with_counties_updated.csv',index=False)

#Removed the word court or courts from the Number of Courts column so it can be used in calculations.
fl_full_info['Number of Courts'] = fl_full_info['Number of Courts'].str.replace(' courts', '')
fl_full_info['Number of Courts'] = fl_full_info['Number of Courts'].str.replace(' court', '')

#Added a column called 'ID' to be used as a primary key.
fl_full_info['ID'] = range(1, len(fl_full_info) + 1)

,Location Name,City,Number of Courts,County,Latitude,Longitude,ID
0,Hal Brady Recreation Complex,Alachua,6,Alachua,29.796996,-82.480531,1
1,Eastmonte Park,Altamonte Springs,4,Seminole,28.662728,-81.371908,2
2,Sanlando Park,Altamonte Springs,10,Seminole,28.662728,-81.371908,3
3,Westmonte Park,Altamonte Springs,3,Seminole,28.662728,-81.371908,4
4,Lake Brantley South Gym,Altamonte Springs,2,Seminole,28.662728,-81.371908,5


In [19]:
#Converted the pickleball dataframe to a SQlite file.
conn = sqlite3.connect('florida_info_db.sqlite')
fl_full_info.to_sql('court_location', conn, index=False, if_exists='replace', dtype={'ID': 'INTEGER PRIMARY KEY'})

1033

In [9]:
#Close the browser.
browser.quit()